# Facial Keypoints Recognition
## Using Sequential Neural Network

In [1]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation,Dropout
import csv as csv

Using Theano backend.
Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 4007)


In [3]:
## Bringing in the data and cleaning it up
train = 'training.csv'
test = 'test.csv'
df = pd.read_csv(train)

In [4]:
df.loc[df['left_eye_center_x'].isnull(),'left_eye_center_x']
## A total of 10 are null in case of the left_eye_center_x

1687   NaN
1834   NaN
1866   NaN
1938   NaN
2100   NaN
2137   NaN
2153   NaN
2175   NaN
2186   NaN
2239   NaN
Name: left_eye_center_x, dtype: float64

In [5]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2140 entries, 0 to 2283
Data columns (total 31 columns):
left_eye_center_x            2140 non-null float64
left_eye_center_y            2140 non-null float64
right_eye_center_x           2140 non-null float64
right_eye_center_y           2140 non-null float64
left_eye_inner_corner_x      2140 non-null float64
left_eye_inner_corner_y      2140 non-null float64
left_eye_outer_corner_x      2140 non-null float64
left_eye_outer_corner_y      2140 non-null float64
right_eye_inner_corner_x     2140 non-null float64
right_eye_inner_corner_y     2140 non-null float64
right_eye_outer_corner_x     2140 non-null float64
right_eye_outer_corner_y     2140 non-null float64
left_eyebrow_inner_end_x     2140 non-null float64
left_eyebrow_inner_end_y     2140 non-null float64
left_eyebrow_outer_end_x     2140 non-null float64
left_eyebrow_outer_end_y     2140 non-null float64
right_eyebrow_inner_end_x    2140 non-null float64
right_eyebrow_inner_end_y 

In [6]:
df['Image'].head()

0    238 236 237 238 240 240 239 241 241 243 240 23...
1    219 215 204 196 204 211 212 200 180 168 178 19...
2    144 142 159 180 188 188 184 180 167 132 84 59 ...
3    193 192 193 194 194 194 193 192 168 111 50 12 ...
4    147 148 160 196 215 214 216 217 219 220 206 18...
Name: Image, dtype: object

In [7]:
df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

In [8]:
df['Image'].head()
X = np.vstack(df['Image'].values)/255
X = X.astype(np.float32)
print X
print np.shape(X)

[[ 0.93333334  0.9254902   0.92941177 ...,  0.27450982  0.29411766
   0.35294119]
 [ 0.85882354  0.84313726  0.80000001 ...,  0.00392157  0.00392157
   0.00392157]
 [ 0.56470591  0.55686277  0.62352943 ...,  0.30588236  0.30588236
   0.3019608 ]
 ..., 
 [ 0.12156863  0.15686275  0.18431373 ...,  0.15294118  0.2         0.29411766]
 [ 0.02745098  0.00392157  0.01960784 ...,  0.7019608   0.69411767
   0.22352941]
 [ 0.26666668  0.07450981  0.07450981 ...,  0.49019608  0.48627451
   0.46666667]]
(2140, 9216)


In [9]:
X_train = X[:1200,:]
X_cv = X[1200:,:]
print np.shape(X_train)
print np.shape(X_cv)

(1200, 9216)
(940, 9216)


In [10]:
y = df[df.columns[:-1]].values
y = y.astype(np.float32)
print np.shape(y)

(2140, 30)


In [11]:
y_train = y[:1200,:]
y_cv = y[1200:,:]
print np.shape(y_train)
print np.shape(y_cv)

(1200, 30)
(940, 30)


## Simple Neural Network Model

In [94]:
model = Sequential([
        Dense(32,input_dim = 9216),
        Activation('relu'),
        Dense(100),
        Activation('relu'),
        Dropout(0.2),
        Dense(30),
        Activation('relu'),
    ])

In [95]:
model.compile(optimizer='adadelta',loss='categorical_crossentropy',metrics=['accuracy'])

In [96]:
model.load_weights('/home/najeeb/Desktop/Dataset/Facial_Keypoints_Recog/Weights.hdf5')

In [97]:
model.fit(X_train,y_train,nb_epoch=400,batch_size=32,verbose=0)

In [98]:
model.save_weights('/home/najeeb/Desktop/Dataset/Facial_Keypoints_Recog/Weights.hdf5',overwrite=True)

In [99]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
dense_4 (Dense)                    (None, 32)          294944      dense_input_2[0][0]              
____________________________________________________________________________________________________
activation_4 (Activation)          (None, 32)          0           dense_4[0][0]                    
____________________________________________________________________________________________________
dense_5 (Dense)                    (None, 100)         3300        activation_4[0][0]               
____________________________________________________________________________________________________
activation_5 (Activation)          (None, 100)         0           dense_5[0][0]                    
___________________________________________________________________________________________

In [21]:
score = model.evaluate(X_cv,y_cv,batch_size = 32)

Exception: Error when checking model input: expected convolution2d_input_1 to have 4 dimensions, but got array with shape (940, 9216)

In [101]:
print score

[4765.4099006815159, 0.52340425531914891]


In [32]:
## Bringing in the test set
df_test = pd.read_csv(test,index_col = 0)
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1783 entries, 1 to 1783
Data columns (total 1 columns):
Image    1783 non-null object
dtypes: object(1)
memory usage: 27.9+ KB


In [33]:
df_test = df_test.dropna()

In [34]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1783 entries, 1 to 1783
Data columns (total 1 columns):
Image    1783 non-null object
dtypes: object(1)
memory usage: 27.9+ KB


In [35]:
df_test.head()

,Image
ImageId,
1,182 183 182 182 180 180 176 169 156 137 124 10...
2,76 87 81 72 65 59 64 76 69 42 31 38 49 58 58 4...
3,177 176 174 170 169 169 168 166 166 166 161 14...
4,176 174 174 175 174 174 176 176 175 171 165 15...
5,50 47 44 101 144 149 120 58 48 42 35 35 37 39 ...


In [36]:
df_test['Image'] = df_test['Image'].apply(lambda im: np.fromstring(im,sep=' '))
df_test['Image']

ImageId
1       [182.0, 183.0, 182.0, 182.0, 180.0, 180.0, 176...
2       [76.0, 87.0, 81.0, 72.0, 65.0, 59.0, 64.0, 76....
3       [177.0, 176.0, 174.0, 170.0, 169.0, 169.0, 168...
4       [176.0, 174.0, 174.0, 175.0, 174.0, 174.0, 176...
5       [50.0, 47.0, 44.0, 101.0, 144.0, 149.0, 120.0,...
6       [177.0, 177.0, 177.0, 171.0, 142.0, 115.0, 97....
7       [77.0, 55.0, 44.0, 56.0, 58.0, 61.0, 67.0, 66....
8       [156.0, 160.0, 162.0, 166.0, 150.0, 114.0, 97....
9       [230.0, 230.0, 231.0, 231.0, 231.0, 231.0, 231...
10      [132.0, 129.0, 126.0, 128.0, 146.0, 163.0, 170...
11      [182.0, 182.0, 182.0, 182.0, 182.0, 181.0, 183...
12      [207.0, 205.0, 204.0, 202.0, 205.0, 197.0, 184...
13      [121.0, 83.0, 58.0, 41.0, 37.0, 36.0, 33.0, 33...
14      [89.0, 60.0, 63.0, 65.0, 65.0, 84.0, 64.0, 35....
15      [88.0, 112.0, 132.0, 132.0, 133.0, 135.0, 131....
16      [153.0, 153.0, 155.0, 156.0, 155.0, 154.0, 153...
17      [97.0, 101.0, 98.0, 92.0, 91.0, 91.0, 95.0, 99...
18    

In [107]:
X_test = np.vstack(df_test['Image'].values)/255

In [108]:
print X_test

[[ 0.71372549  0.71764706  0.71372549 ...,  0.03137255  0.01960784
   0.01960784]
 [ 0.29803922  0.34117647  0.31764706 ...,  0.89411765  0.82745098
   0.65882353]
 [ 0.69411765  0.69019608  0.68235294 ...,  0.00392157  0.00392157
   0.00392157]
 ..., 
 [ 0.10980392  0.10980392  0.11372549 ...,  0.30196078  0.30588235
   0.30588235]
 [ 0.40784314  0.37254902  0.27843137 ...,  0.56470588  0.59215686
   0.62352941]
 [ 0.24705882  0.23921569  0.25098039 ...,  0.44313725  0.43921569
   0.44313725]]


In [109]:
yPred = model.predict(X_test,batch_size=32,verbose = 0)

In [110]:
print yPred
print np.shape(yPred)

[[ 14.05997181   7.81734133   5.98451424 ...,  14.54801846  10.20882607
   17.73294449]
 [  9.59679127   5.1310401    3.96576643 ...,  10.67988396   7.13038301
   12.54106331]
 [ 17.5809536    9.73304844   8.34363651 ...,  19.10345459  13.63756752
   22.02163315]
 ..., 
 [  7.81345463   4.07394934   3.45347261 ...,   8.14753437   5.3670783
    9.71312904]
 [ 10.09251308   5.88532495   4.4658947  ...,  10.29162216   7.48822737
   12.74537945]
 [  7.54152155   4.34356165   3.37557602 ...,   8.11832333   5.26921129
    9.53826237]]
(1783, 30)


In [111]:
for i in range(np.shape(yPred)[0]):
    for j in range(np.shape(yPred)[1]):
        if yPred[i][j] < 0:
            yPred[i][j] = 0.0

## Using Convolution Neural Networks

In [12]:
## Takes a long time to run so use a GPU
from keras.layers import Convolution2D,MaxPooling2D,Flatten

In [13]:
## Reshaping the data into 3D format
print X_train.shape
print y_train.shape
X_train = np.reshape(X_train,(X_train.shape[0],1,96,96))
print X_train.shape

(1200, 9216)
(1200, 30)
(1200, 1, 96, 96)


In [14]:
model = Sequential()
## First Convolution layer
model.add(Convolution2D(32,3,3,border_mode = 'valid', input_shape = (1,96,96)))
model.add(Activation('relu'))
model.add(Convolution2D(32,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
## Second Convolution Layer
model.add(Convolution2D(64,3,3,border_mode = 'valid'))
model.add(Activation('relu'))
model.add(Convolution2D(64,3,3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.25))
## Adding the fully connected layer
model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
## Addding the final ouput layer
model.add(Dense(30))
model.add(Activation('relu'))

In [25]:
model.compile(loss = 'mse',optimizer = 'adagrad',metrics = ['accuracy'])

In [26]:
model.load_weights('Weights_CNN.hdf5')

In [27]:
model.fit(X_train,y_train,nb_epoch=50,batch_size=32,verbose = 1)

Epoch 1/50
1200/1200 [==============================] - 5s - loss: 18505.2442 - acc: 0.5933     
Epoch 2/50
1200/1200 [==============================] - 5s - loss: 128.1210 - acc: 0.4617     
Epoch 3/50
1200/1200 [==============================] - 5s - loss: 89.1057 - acc: 0.4550     
Epoch 4/50
1200/1200 [==============================] - 5s - loss: 88.5229 - acc: 0.4700     
Epoch 5/50
1200/1200 [==============================] - 5s - loss: 77.2224 - acc: 0.4950     
Epoch 6/50
1200/1200 [==============================] - 5s - loss: 74.2620 - acc: 0.5033     
Epoch 7/50
1200/1200 [==============================] - 5s - loss: 68.6746 - acc: 0.5242     
Epoch 8/50
1200/1200 [==============================] - 6s - loss: 66.2490 - acc: 0.5192     
Epoch 9/50
1200/1200 [==============================] - 5s - loss: 65.3634 - acc: 0.5425     
Epoch 10/50
1200/1200 [==============================] - 5s - loss: 64.4533 - acc: 0.5417     
Epoch 11/50
1200/1200 [==============================] 

In [20]:
model.save_weights('Weights_CNN.hdf5')

[WARNING] Weights_CNN.hdf5 already exists - overwrite? [y/n]y
[TIP] Next time specify overwrite=True in save_weights!


In [28]:
print X_cv.shape
print y_cv.shape
X_cv = np.reshape(X_cv,(X_cv.shape[0],1,96,96))
print X_cv.shape

(940, 1, 96, 96)
(940, 30)
(940, 1, 96, 96)


In [29]:
score = model.evaluate(X_cv,y_cv,batch_size = 32)

940/940 [==============================] - 0s     


In [30]:
print score

[41.143131905413689, 0.52553191489361706]


In [37]:
X_test = np.vstack(df_test['Image'].values)/255
X_test = np.reshape(X_test,(X_test.shape[0],1,96,96))

In [38]:
yPred = model.predict(X_test,batch_size = 32,verbose = 1)

1783/1783 [==============================] - 1s     


In [39]:
print yPred

[[ 60.24824524  33.65006638  27.05023003 ...,  64.10475159  43.48781204
   76.24269104]
 [ 63.12611008  35.42623901  28.45466423 ...,  67.37026978  45.73833084
   79.98352814]
 [ 62.29702759  35.07566452  28.22402763 ...,  66.61660767  45.31272507
   78.83598328]
 ..., 
 [ 58.62841797  33.05129242  26.5137043  ...,  62.38238144  42.5090332
   74.02697754]
 [ 59.52952194  33.33998871  26.77518082 ...,  63.24361038  43.01625443
   75.15684509]
 [ 63.23072433  35.15843964  28.12704277 ...,  66.98067474  45.45367432
   80.3243103 ]]


In [50]:
for row in yPred:
    for value in row:
        if value < 0.0 or value > 95.0:
            value = 0.0

## Writing the data into the file

In [45]:
## Writing the data into a csv file
fp = open('submission.csv',"w")
p = csv.writer(fp)
fp_lookup = open('IdLookupTable.csv',"r")
lookup = csv.reader(fp_lookup)
header = lookup.next()

In [46]:
features = []
for feature in df.columns[:]:
    features.append(feature)
features.pop(-1)
print len(features)

30


In [47]:
## Creating a dictionary of features
h = dict((w,i) for i,w in enumerate(features))

In [48]:
p.writerow(['RowId','Location'])
for row in lookup:
    p.writerow([row[0],yPred[int(row[1]) - 1][h[row[2]]]])
fp.close()
fp_lookup.close()